In [2]:
import numpy as np
from scipy.sparse import csr_matrix as csr
from scipy.sparse import eye
import random

In [3]:
from prettytable import PrettyTable

def print_matrix(A):
    k, n = A.shape
    p = PrettyTable()
    for i in range(k):
        for j in range(k):
            A[i][j] = round(A[i][j], 3)
        p.add_row(A[i])
    print(p.get_string(header=False, border=False))
    print()

In [4]:
pip install PrettyTable

Note: you may need to restart the kernel to use updated packages.


# LU Decomposition

In [5]:
def LU_decompse(A): # LU разложение Вход 2х мерный массив, Выход 2х мерные массивы
    n_rows = len(A)
    n_cols = len(A[0])
    L = np.eye(n_rows, n_cols)
    U = np.zeros((n_rows, n_cols))
    for i in range(n_rows):
        for j in range(n_cols):
            if i <= j:
                _s = 0
                for k in range(i):
                    _s += L[i][k] * U[k][j]
                U[i][j] = A[i][j] - _s
            else:
                _s = 0
                for k in range(j):
                    _s += L[i][k] * U[k][j]
                L[i][j] = (A[i][j] - _s) / U[j][j]
    return L, U

In [6]:
def LU_decompse_csr(A): # LU разложение Вход csr матрица Выход csr матрица (не оптимально)
    n_rows, n_cols = A.shape
    L = eye(n_rows, n_cols, format="csr")
    U = csr((n_rows, n_cols), dtype=float)
    for i in range(n_rows):
        for j in range(n_cols):
            if i <= j:
                _s = 0
                for k in range(i):
                    _s += L[i, k] * U[k, j]
                U[i, j] = A[i, j] - _s
            else:
                _s = 0
                for k in range(j):
                    _s += L[i, k] * U[k, j]
                L[i, j] = (A[i, j] - _s) / U[j, j]
    return L, U

In [7]:
def LU_decompse_csr_full(A):
    n_rows, n_cols = A.shape
    l_data = []
    l_indices = []
    l_indptr = []
    u_data = []
    u_indices = []
    u_indptr = []
    for i in range(n_rows):
        l_indptr.append(len(l_data))
        u_indptr.append(len(u_data))
        for j in range(n_cols):
            if i <= j:
                _s = 0
                if i == j:
                    l_data.append(1)
                    l_indices.append(j)
                for x, y in zip(l_data[l_indptr[i]:], list(map(lambda x: x[0], filter(lambda x: x[1] == j, zip(u_data, u_indices))))):
                    _s += x * y
                u_data.append(A[i, j] - _s)
                u_indices.append(j)
            else:
                _s = 0
                for x, y in zip(l_data[l_indptr[i]:], list(map(lambda x: x[0], filter(lambda x: x[1] == j, zip(u_data, u_indices))))):
                    _s += x * y
                l_data.append((A[i, j] - _s) / u_data[u_indptr[j]])
                l_indices.append(j)
    l_indptr.append(len(l_data))
    u_indptr.append(len(u_data))
    return csr((l_data, l_indices, l_indptr)), csr((u_data, u_indices, u_indptr))

## Example

In [8]:
A = np.array([
    [6, -3, 5, 0, 2, 0, 0],
    [-4, 0, 7, -3, 0, 2, 0],
    [0, 9, -3, -6, 0, 7, 1],
    [5, -2, 0, 0, 1, 7, -3],
    [-1, 0, 0, 5, 0, 2, 0],
    [9, -8, 7, 0, 2, 3, 0],
    [3, 0, -4, 1, 9, 0, 5]
])
L, U = LU_decompse(A)
print("Matrix A")
print_matrix(A)
print("Matrix L")
print_matrix(L)
print("Matrix U")
print_matrix(U)
B = np.dot(L, U)
print("Matrix L * U")
print_matrix(B)

Matrix A
 6   -3  5   0   2  0  0  
 -4  0   7   -3  0  2  0  
 0   9   -3  -6  0  7  1  
 5   -2  0   0   1  7  -3 
 -1  0   0   5   0  2  0  
 9   -8  7   0   2  3  0  
 3   0   -4  1   9  0  5  

Matrix L
  1.0     0.0    0.0     0.0      0.0      0.0    0.0 
 -0.667   1.0    0.0     0.0      0.0      0.0    0.0 
  0.0     -4.5   1.0     0.0      0.0      0.0    0.0 
 0.833   -0.25  -0.036   1.0      0.0      0.0    0.0 
 -0.167   0.25  -0.04   -3.402    1.0      0.0    0.0 
  1.5     1.75  -0.427   2.11    3.526     1.0    0.0 
  0.5    -0.75  0.029   0.472   -59.368  -15.245  1.0 

Matrix U
 6.0  -3.0   5.0     0.0    2.0      0.0       0.0   
 0.0  -2.0  10.333   -3.0  1.333     2.0       0.0   
 0.0  0.0    43.5   -19.5   6.0      16.0      1.0   
 0.0  0.0    0.0    -1.46  -0.115   8.082     -2.964 
 0.0  0.0    0.0     0.0   -0.15    29.636   -10.041 
 0.0  0.0    0.0     0.0    0.0    -115.228   42.088 
 0.0  0.0    0.0     0.0    0.0      0.0      51.914 

Matrix L * U
  6.0

In [9]:
A = np.array([
    [6, -3, 5, 0, 2, 0, 0],
    [-4, 0, 7, -3, 0, 2, 0],
    [0, 9, -3, -6, 0, 7, 1],
    [5, -2, 0, 0, 1, 7, -3],
    [-1, 0, 0, 5, 0, 2, 0],
    [9, -8, 7, 0, 2, 3, 0],
    [3, 0, -4, 1, 9, 0, 5]
])
L, U = LU_decompse_csr_full(A)
print(L.toarray())

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-6.66666667e-01  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -4.50000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 8.33333333e-01 -2.50000000e-01 -3.63984674e-02  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.66666667e-01  2.50000000e-01 -4.02298851e-02 -3.40157480e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.50000000e+00  1.75000000e+00 -4.27203065e-01  2.11023622e+00
   3.52631579e+00  1.00000000e+00  0.00000000e+00]
 [ 5.00000000e-01 -7.50000000e-01  2.87356322e-02  4.72440945e-01
  -5.93684211e+01 -1.52453563e+01  1.00000000e+00]]


In [10]:
A = np.array([[10, -7.6666, 0], 
              [-3, 6, 2], 
              [5, -1, 5]])

print_matrix(A)
# sA = csr(A)
# print("sA")
# print(sA)

# print(LU_decompse(A))
# l, u =  LU_decompse_csr(sA)
# print("L: ", l)
# print("U: ", u)

# l, u = LU_decompse_csr_full(sA)
# print("L: ", l.toarray())
# print("U: ", u.toarray())

 10.0  -7.667  0.0 
 -3.0   6.0    2.0 
 5.0    -1.0   5.0 



# Usage of sparse matrices

In [11]:
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
mtx = csr((data, (row, col)), shape=(3, 3))
mtx     

<3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [12]:
mtx.todense()

matrix([[1, 0, 2],
        [0, 0, 3],
        [4, 5, 6]])

In [13]:
mtx.data

array([1, 2, 3, 4, 5, 6])

In [14]:
mtx.indices

array([0, 2, 2, 0, 1, 2], dtype=int32)

In [15]:
mtx.indptr

array([0, 2, 3, 6], dtype=int32)

# Solving a system of linear equations

## Additional functions

In [16]:
# get i-th row of sparse matrix A

def get_row(A, i):
    _, n = A.shape
    row = np.zeros(n)
    non_zero_elements_in_row = A.data[A.indptr[i]:A.indptr[i + 1]]
    indices_of_non_zero_elements_in_row = A.indices[A.indptr[i]:A.indptr[i + 1]]
    k = 0
    for i in range(n):
        if k < len(non_zero_elements_in_row) and i == indices_of_non_zero_elements_in_row[k]:
            row[i] = non_zero_elements_in_row[k]
            k += 1
    return row

In [17]:
# get matrix norm

def norm(A):
    result = 0
    k, n = A.shape
    for i in range(k):
        for j in range(n):
            result += A[i][j] ** 2
    return np.sqrt(result)

In [18]:
# get conditional number of matrix

def get_conditional_number(A):
    A_inverse = inverse(A).todense()
    A_inverse = np.squeeze(np.asarray(A_inverse))
    return norm(A) * norm(A_inverse)

## Solve function

In [19]:
# A: sparse matrix
# b: array of values

def solve(A, b):
    L, U = LU_decompse_csr_full(A)
    _, n = A.shape
    
    # solving Ly = b
    y = np.zeros(n)
    for i in range(n):
        row = get_row(L, i)
        y[i] = b[i] - np.dot(y[:i], row[:i])
    
    # solving Ux = y
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        row = get_row(U, i)
        x[i] = (y[i] - np.dot(x[i + 1:], row[i + 1:])) / row[i]
        
    return x

## Example

In [20]:
A = np.array([[10, -7, 2, -3, 0, 0], 
              [-3, 4, 6, 0, 5, 4], 
              [5, -1, 5, -9, 2, 8],
              [0, 1, -6, 3, 2, 7],
              [0, 0, 0, 5, -9, 3],
              [6, 0, 4, 7, -2, 0]
])
b = [-15, 23, -3, -34, 27, 8]
x = solve(csr(A), b)
print("Matrix A")
print_matrix(A)
print("Vector b\n", b, "\n")
print("Vector x\n ", x, "\n")
print("Vector A * x")
print(np.dot(A, x))

Matrix A
 10  -7  2   -3  0   0 
 -3  4   6   0   5   4 
 5   -1  5   -9  2   8 
 0   1   -6  3   2   7 
 0   0   0   5   -9  3 
 6   0   4   7   -2  0 

Vector b
 [-15, 23, -3, -34, 27, 8] 

Vector x
  [-4.17801592 -2.74871319  5.30844092  1.02590499 -2.32649847  0.31066295] 

Vector A * x
[-15.  23.  -3. -34.  27.   8.]


# Finding inverse matrix

In [21]:
def inverse(A):
    _, n = A.shape
    I = np.eye(n)
    A_inverse = np.zeros(A.shape)
    for i in range(n):
        x = solve(A, I[:, i])
        A_inverse[:, i] = x
    return csr(A_inverse)

## Example

In [22]:
A = np.array([
    [3, -5, 0, 2, 0, 8],
    [4, 1, 2, 0, 4, 3],
    [1, -1, 3, 0, 2, 0],
    [0, 1, 9, -2, 5, 0],
    [7, -4, -5, 0, 6, -3],
    [0, -4, 2, 8, 0, 7]
])

np.set_printoptions(precision=3)

print("Matrix A")
print_matrix(A)
print("Inverse Matrix A")
A_inverse = inverse(csr(A)).todense()
print(A_inverse, "\n")
print("Matrix E")
E = np.dot(A, A_inverse)
print(E)

Matrix A
 3  -5  0   2   0  8  
 4  1   2   0   4  3  
 1  -1  3   0   2  0  
 0  1   9   -2  5  0  
 7  -4  -5  0   6  -3 
 0  -4  2   8   0  7  

Inverse Matrix A
[[-0.065  0.275  1.12  -0.492 -0.147 -0.107]
 [-0.117  0.218 -0.04  -0.076 -0.069  0.01 ]
 [-0.034  0.06   0.647 -0.172 -0.112 -0.034]
 [-0.149  0.083  0.266 -0.147 -0.022  0.126]
 [ 0.026 -0.119 -1.05   0.467  0.207  0.11 ]
 [ 0.114  0.012 -0.511  0.174  0.017  0.015]] 

Matrix E
[[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  5.551e-17]
 [ 1.110e-16  1.000e+00  4.441e-16 -2.220e-16 -4.163e-17  2.776e-17]
 [ 1.388e-17 -2.776e-17  1.000e+00 -2.776e-17  4.163e-17  1.735e-18]
 [ 1.110e-16 -3.469e-17  3.331e-16  1.000e+00 -6.939e-17 -5.551e-17]
 [ 4.996e-16 -2.220e-16 -1.776e-15  4.441e-16  1.000e+00  3.331e-16]
 [ 6.800e-16 -5.135e-16  1.332e-15 -3.220e-15 -3.886e-16  1.000e+00]]


# Diagonal Matrices

In [23]:
random.seed()
def generate_diagonal_matrix(k):
    values = [0, -1, -2, -3, -4, -5, -6]
    noise = 10**(-k)
    matrix = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            matrix[i][j] = random.choice(values)
    for i in range(k):
        matrix[i][i] = -(sum(matrix[i]) - matrix[i][i]) + noise
    return matrix

def solve_d_system(k):
    A = generate_diagonal_matrix(k)
#     while abs(np.linalg.det(A)) < 10**(-5):
#         A = generate_diagonal_matrix(k)
    x = [x for x in range(1, k + 1)]
    F = np.dot(A, x)
    x_new = solve(csr(A), F)
    error = np.linalg.norm(x - x_new)
    return x_new, error

def solve_d_systems(k):
    error_array = []
    x_new_array = []
    for i in range(1, k + 1):
        x_new, error = solve_d_system(i)
        x_new_array.append(x_new)
        error_array.append(error)
        print("k =", i, "\terror =", round(error, 5), "\tx' =", x_new)

        
solve_d_systems(15)

k = 1 	error = 0.0 	x' = [1.]
k = 2 	error = 0.0 	x' = [1. 2.]
k = 3 	error = 0.0 	x' = [1. 2. 3.]
k = 4 	error = 0.0 	x' = [1. 2. 3. 4.]
k = 5 	error = 0.0 	x' = [1. 2. 3. 4. 5.]
k = 6 	error = 0.0 	x' = [1. 2. 3. 4. 5. 6.]
k = 7 	error = 0.0 	x' = [1. 2. 3. 4. 5. 6. 7.]
k = 8 	error = 0.0 	x' = [1. 2. 3. 4. 5. 6. 7. 8.]
k = 9 	error = 0.0 	x' = [1. 2. 3. 4. 5. 6. 7. 8. 9.]
k = 10 	error = 3e-05 	x' = [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
k = 11 	error = 0.00096 	x' = [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
k = 12 	error = 0.00654 	x' = [ 0.998  1.998  2.998  3.998  4.998  5.998  6.998  7.998  8.998  9.998
 10.998 11.998]
k = 13 	error = 0.19667 	x' = [ 1.055  2.055  3.055  4.055  5.055  6.055  7.055  8.055  9.055 10.055
 11.055 12.055 13.055]
k = 14 	error = 5.72253 	x' = [ 2.529  3.529  4.529  5.529  6.529  7.529  8.529  9.529 10.529 11.529
 12.529 13.529 14.529 15.529]
k = 15 	error = 42.60282 	x' = [-1.000e+01 -9.000e+00 -8.000e+00 -7.000e+00 -6.000e+00 -5.000e+00
 -4.00

# Gilbert Matrices

In [24]:
def generate_gilbert_matrix(k):
    matrix = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            matrix[i][j] = 1 / (i + j + 1)
    return matrix


def solve_system(k):
    A = generate_gilbert_matrix(k)
    x = np.zeros((k))
    for i in range(k):
        x[i] = i + 1
    F = np.dot(A, x)
    x_new = solve(csr(A), F)
    error = np.linalg.norm(x - x_new)
    conditional_number = get_conditional_number(A)
    return x_new, error, conditional_number
    

def solve_systems(k):
    error_array = []
    x_new_array = []
    conditional_number_array = []
    for i in range(2, k + 1):
        x_new, error, conditional_number = solve_system(i)
        x_new_array.append(x_new)
        error_array.append(error)
        conditional_number_array.append(conditional_number)
        print("k =", i, "\terror =", round(error, 5), "\tconditional number =", conditional_number, "\nx' =", x_new, "\n")
    return error_array, x_new_array, conditional_number_array
    

solve_systems(15)

k = 2 	error = 0.0 	conditional number = 19.333333333333336 
x' = [1. 2.] 

k = 3 	error = 0.0 	conditional number = 526.1588210797213 
x' = [1. 2. 3.] 

k = 4 	error = 0.0 	conditional number = 15613.79355963998 
x' = [1. 2. 3. 4.] 

k = 5 	error = 0.0 	conditional number = 480849.1169937436 
x' = [1. 2. 3. 4. 5.] 

k = 6 	error = 0.0 	conditional number = 15118987.129212683 
x' = [1. 2. 3. 4. 5. 6.] 

k = 7 	error = 0.0 	conditional number = 481747256.05358005 
x' = [1. 2. 3. 4. 5. 6. 7.] 

k = 8 	error = 0.0 	conditional number = 15493619331.329567 
x' = [1. 2. 3. 4. 5. 6. 7. 8.] 

k = 9 	error = 0.00016 	conditional number = 501729608115.98737 
x' = [1. 2. 3. 4. 5. 6. 7. 8. 9.] 

k = 10 	error = 0.00041 	conditional number = 16331278452337.033 
x' = [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.] 

k = 11 	error = 0.0345 	conditional number = 532286585396228.3 
x' = [ 1.     2.     3.     4.     4.998  6.008  6.984  8.022  8.982 10.008
 10.998] 

k = 12 	error = 3.89577 	conditional numb

([1.4895204919483639e-15,
  1.4895204919483639e-15,
  2.430119701128671e-12,
  3.507528484380447e-11,
  3.5876912959877836e-09,
  9.513819437681708e-08,
  1.3291634883106896e-06,
  0.0001644389083877056,
  0.000406020022566205,
  0.034503490462739116,
  3.8957725013405997,
  357.8527735697525,
  195.9230395442151,
  242.31330546983443],
 [array([1., 2.]),
  array([1., 2., 3.]),
  array([1., 2., 3., 4.]),
  array([1., 2., 3., 4., 5.]),
  array([1., 2., 3., 4., 5., 6.]),
  array([1., 2., 3., 4., 5., 6., 7.]),
  array([1., 2., 3., 4., 5., 6., 7., 8.]),
  array([1., 2., 3., 4., 5., 6., 7., 8., 9.]),
  array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
  array([ 1.   ,  2.   ,  3.   ,  4.   ,  4.998,  6.008,  6.984,  8.022,
          8.982, 10.008, 10.998]),
  array([ 1.   ,  2.   ,  2.999,  4.013,  4.907,  6.41 ,  5.861, 10.059,
          6.59 , 11.762, 10.268, 12.132]),
  array([   1.   ,    2.001,    2.98 ,    4.334,    2.01 ,   22.194,
          -49.432,  138.741, -194.465,  220